In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv('data.csv')

In [11]:
# import numpy as np
# import pandas as pd

def RSI(dataframe, period):
    '''
    Computes the RSI of a given price series for a given period length
    :param dataframe:
    :param period:
    :return dataframe with rsi:
    '''

    # TODO:---
    # for symbol in dataframe:
    #     all_prices = dataframe[symbol].close
    # ---

    # dataframe["RSI"] = dataframe.apply(lambda row: RSI)
    rsi = []

    for stock in dataframe['Symbol'].unique():
        all_prices = dataframe[dataframe['Symbol'] == stock]['Close']
        diff = np.diff(all_prices) # length is 1 less than the all_prices
        for i in range(period):
            rsi.append(None) # because RSI can't be calculated until period prices have occured

        for i in range(len(diff) - period + 1):
            avg_gain = diff[i:period + i]
            avg_loss = diff[i:period + i]
            avg_gain = abs(sum(avg_gain[avg_gain >= 0]) / period)
            avg_loss = abs(sum(avg_loss[avg_loss < 0]) / period)
            if avg_loss == 0:
                rsi.append(100)
            elif avg_gain == 0:
                rsi.append(0)
            else:
                rs = avg_gain / avg_loss
                rsi.append(100 - (100 / (1 + rs)))

    dataframe['RSI'] = rsi
    return dataframe


def PROC(dataframe, period):
    '''
    Computes the PROC(price rate of change) of a given price series for a given period length
    :param dataframe:
    :param period:
    :return proc:
    '''

    
    proc = []

    for stock in dataframe['Symbol'].unique():
        all_prices = list(dataframe[dataframe['Symbol'] == stock]['Close'])
        for i in range(period):
            proc.append(None) # because proc can't be calculated until period prices have occured
#         print(all_prices, "THEN LEN", len(all_prices))
#         print(all_prices[2])
        for i in range(len(all_prices) - period):
            if len(all_prices) <= period:
#                 print("This shouldn't be happening lololol", len(all_prices))
                proc.append(None)
            else:
#                 print("STOCK:", stock)
#                 print("INDEX:", i)
#                 print("LENGTH:", len(all_prices))
#                 print("WHAT WE HAVE SOFAR (LAST 3):", proc[-3:])
                proc.append((all_prices[i + period] - all_prices[i]) / all_prices[i])

    dataframe['PROC'] = proc
    return dataframe

#computes the stochastic oscillator 
def s_o(close, low, high):
    return 100 * (close - low)/(high - low)

def SO(df, intervall):
    
    df['SO'] = np.nan
    high = df.ix[1, 'High']
    low = df.ix[1, 'Low']
    lasth = 0
    lastl = 0
    
    for i in range(1,len(df)):
        
        if lasth < i - intervall or high <= df.ix[i, 'High']:
            high = np.max(df['High'][i-intervall:i])
            lasth = i
        if lastl < i - intervall or low <= df.ix[i, 'Low']:
            low = np.min(df['Low'][i-intervall:i])
            lastl = i
        
        if df.ix[i, 'Date'] != df.ix[(i-1), 'Date']:
            df.ix[i, 'SO'] = s_o(df.ix[i, 'Close'], low, high)
        else:
            df.ix[i, 'SO'] = df.ix[i-1, 'SO']
    return df

def Williams_R(dataframe, period):
    '''
    Williams %R
    Calculates fancy shit for late usage. Nice!

    EXAMPLE USAGE:
    data = pandas.read_csv("./data/ALL.csv", sep=",",header=0,quotechar='"')
    wr = Williams_R(data)
    print(wr)

    '''
    wr = []
    for stock in dataframe['Symbol'].unique():
        all_prices = list(dataframe[dataframe['Symbol'] == stock]['Close'])
        for i in range(period):
            wr.append(None) # because proc can't be calculated until period prices have occured
            
        for i in range(period-1,len(all_prices)-1):
            C = all_prices[i]
            H = max(all_prices[i-period+1:i])
            L = min(all_prices[i-period+1:i])
            wr_one = (
                ((H - C) 
                 / (H - L)) * -100
            )
            if wr_one <=-100:
                wr.append(-100)
            else:
                wr.append(wr_one)
    dataframe["WR"] = wr
    return dataframe

def On_Balance_Volume(dataframe):
    '''
    Williams %R
    Calculates fancy shit for late usage. Nice!

    EXAMPLE USAGE:
    data = pandas.read_csv("./data/ALL.csv", sep=",",header=0,quotechar='"')
    wr = Williams_R(data)
    print(wr)

    '''
    obv = []
    obv.append(dataframe.iloc[0]["Volume"])
    for i in range(1,len(dataframe)):
        C_old = dataframe.iloc[i-1]["Close"]
        C = dataframe.iloc[i]["Close"]
        if(C > C_old):
            obv.append(obv[i-1]+dataframe.iloc[i]["Volume"])
        elif (C < C_old):
            obv.append(obv[i - 1] - dataframe.iloc[i]["Volume"])
        else:
            obv.append(obv[i-1])

    return obv

# data = pd.read_csv("./data/ALL.csv", sep=",",header=0,quotechar='"')
# print(data)
# obv = On_Balance_Volume(data)
# print(obv)


In [3]:
data = PROC(data, 14)
print("PROC: Done")
data = RSI(data,14)
print("RSI: Done")
#data = SO(data,3)
#print("SO: Done")

/usr/lib/python3/dist-packages/ipykernel/__main__.py:69: RuntimeWarning: invalid value encountered in double_scalars
/usr/lib/python3/dist-packages/ipykernel/__main__.py:69: RuntimeWarning: divide by zero encountered in double_scalars


PROC: Done
RSI: Done


In [12]:
data = Williams_R(data, 14 )
print("Williams_R: Done")

/usr/lib/python3/dist-packages/ipykernel/__main__.py:124: RuntimeWarning: divide by zero encountered in double_scalars
/usr/lib/python3/dist-packages/ipykernel/__main__.py:124: RuntimeWarning: invalid value encountered in double_scalars


Williams_R: Done


In [13]:
print(data.head(300))

           Date   Open   High    Low  Close      Volume Symbol          WR
0    2010-01-04  31.39  31.63  31.13  31.30   2729240.0      A         NaN
1    2010-01-05  31.21  31.22  30.76  30.96   2994237.0      A         NaN
2    2010-01-06  30.85  31.00  30.76  30.85   2320299.0      A         NaN
3    2010-01-07  30.78  30.82  30.50  30.81   2213927.0      A         NaN
4    2010-01-08  30.64  30.85  30.40  30.80   2670856.0      A         NaN
5    2010-01-11  30.88  31.05  30.67  30.82   3420258.0      A         NaN
6    2010-01-12  30.56  30.65  30.22  30.45   2053677.0      A         NaN
7    2010-01-13  30.47  30.78  30.05  30.69   2445558.0      A         NaN
8    2010-01-14  30.59  31.24  30.50  31.15   4408930.0      A         NaN
9    2010-01-15  31.22  31.36  30.33  30.43   3309434.0      A         NaN
10   2010-01-19  30.36  30.83  30.35  30.80   2549092.0      A         NaN
11   2010-01-20  30.53  30.67  30.19  30.63   3282598.0      A         NaN
12   2010-01-21  31.00  3

In [29]:
def Williams_R(dataframe, period):
    '''
    Williams %R
    Calculates fancy shit for late usage. Nice!

    EXAMPLE USAGE:
    data = pandas.read_csv("./data/ALL.csv", sep=",",header=0,quotechar='"')
    wr = Williams_R(data)
    print(wr)

    '''
    wr = []
    for stock in dataframe['Symbol'].unique():
        all_prices = list(dataframe[dataframe['Symbol'] == stock]['Close'])
        for i in range(period):
            wr.append(None) # because proc can't be calculated until period prices have occured
            
        for i in range(period-1,len(all_prices)-1):
            C = all_prices[i]
            H = max(all_prices[i-period+1:i])
            L = min(all_prices[i-period+1:i])
            wr.append(
                ((H - C) 
                 / (H - L)) * -100
            )
        return wr
    dataframe["WR"] = wr
    return dataframe

In [3]:
wr = Williams_R(data,14)
print(wt)

/usr/lib/python3/dist-packages/ipykernel/__main__.py:124: RuntimeWarning: divide by zero encountered in double_scalars
/usr/lib/python3/dist-packages/ipykernel/__main__.py:124: RuntimeWarning: invalid value encountered in double_scalars


NameError: name 'wt' is not defined

In [6]:
print(wr)

              Date   Open   High    Low  Close     Volume Symbol          WR
0       2010-01-04  31.39  31.63  31.13  31.30  2729240.0      A         NaN
1       2010-01-05  31.21  31.22  30.76  30.96  2994237.0      A         NaN
2       2010-01-06  30.85  31.00  30.76  30.85  2320299.0      A         NaN
3       2010-01-07  30.78  30.82  30.50  30.81  2213927.0      A         NaN
4       2010-01-08  30.64  30.85  30.40  30.80  2670856.0      A         NaN
5       2010-01-11  30.88  31.05  30.67  30.82  3420258.0      A         NaN
6       2010-01-12  30.56  30.65  30.22  30.45  2053677.0      A         NaN
7       2010-01-13  30.47  30.78  30.05  30.69  2445558.0      A         NaN
8       2010-01-14  30.59  31.24  30.50  31.15  4408930.0      A         NaN
9       2010-01-15  31.22  31.36  30.33  30.43  3309434.0      A         NaN
10      2010-01-19  30.36  30.83  30.35  30.80  2549092.0      A         NaN
11      2010-01-20  30.53  30.67  30.19  30.63  3282598.0      A         NaN

In [57]:
print(data.head(5000))

            Date   Open   High    Low  Close       Volume Symbol        RSI  \
0     2010-01-04  31.39  31.63  31.13  31.30    2729240.0      A        NaN   
1     2010-01-05  31.21  31.22  30.76  30.96    2994237.0      A        NaN   
2     2010-01-06  30.85  31.00  30.76  30.85    2320299.0      A        NaN   
3     2010-01-07  30.78  30.82  30.50  30.81    2213927.0      A        NaN   
4     2010-01-08  30.64  30.85  30.40  30.80    2670856.0      A        NaN   
5     2010-01-11  30.88  31.05  30.67  30.82    3420258.0      A        NaN   
6     2010-01-12  30.56  30.65  30.22  30.45    2053677.0      A        NaN   
7     2010-01-13  30.47  30.78  30.05  30.69    2445558.0      A        NaN   
8     2010-01-14  30.59  31.24  30.50  31.15    4408930.0      A        NaN   
9     2010-01-15  31.22  31.36  30.33  30.43    3309434.0      A        NaN   
10    2010-01-19  30.36  30.83  30.35  30.80    2549092.0      A        NaN   
11    2010-01-20  30.53  30.67  30.19  30.63    3282